In [3]:
import json

In [1]:
file_path = './tests/memory/'

In [2]:
file_name = '000-addi.hex'

# Reading the contents of the file
with open(file_path+file_name, 'r') as file:
    hex_content = file.readlines()

# Display the first few lines of the file to understand its format
hex_content[:10]

['00500593\n', '02b02e23\n']

In [ ]:
# Dictionary for opcode to instruction type mapping (partial list for demo)
opcode_map = {
    '0110011': 'R-type',  # Arithmetic register-register
    '0010011': 'I-type',  # Arithmetic immediate
    '0000011': 'I-type',  # Load
    '0100011': 'S-type',  # Store
    '1100011': 'B-type',  # Branch
    '1101111': 'J-type',  # Jump
    '0110111': 'U-type',  # LUI
    '0010111': 'U-type'   # AUIPC
}

In [14]:
def hex_to_bin(hex_str):
    return bin(int(hex_str, 16))[2:].zfill(32)

In [15]:
# Function to decode instructions
def decode_instruction(hex_instr):
    bin_instr = hex_to_bin(hex_instr) 
    opcode = bin_instr[-7:]
    instr_type = opcode_map.get(opcode, 'Unknown')
    return bin_instr, opcode, instr_type

In [13]:
# Testing with the first two instructions from the file
decoded_instructions = []
for hex_instr in hex_content[:2]:
    hex_instr = hex_instr.strip()  # Removing newline characters
    decoded_instructions.append(decode_instruction(hex_instr))

decoded_instructions  # Display the decoded results

[('00000000010100000000010110010011', '0010011', 'I-type'),
 ('00000010101100000010111000100011', '0100011', 'S-type')]